In [5]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import warnings

In [3]:
df = pd.read_csv('data\StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [11]:
new_cols = [col.replace(' ','_').replace('/','_') for col in df.columns]
new_cols

df.columns = new_cols

In [14]:
X= df.drop('math_score', axis = 1)
y = df['math_score']

In [17]:
numerical_features = [feature for feature in X.columns if X[feature].dtype != 'object']
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

numeric_transformer = StandardScaler()
object_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [("OneHotEncoder", object_transformer, categorical_features),
    ("StandardScaler", numeric_transformer, numerical_features)]
)

In [19]:
X = pd.DataFrame(preprocessor.fit_transform(X))
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.193999,0.391492
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.427476,1.313269
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.770109,1.642475
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.833899,-1.583744
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.605158,0.457333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2.044215,1.774157
996,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.970952,-0.859491
997,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.125472,-0.201079
998,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.605158,0.589015


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [26]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return [mae, rmse, r2_square]

In [22]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}


for name, model in models.items():
    model.fit(X_train, y_train)
    print(f'{name} trained!')

Linear Regression trained!
Lasso trained!
Ridge trained!
K-Neighbors Regressor trained!
Decision Tree trained!
Random Forest Regressor trained!
XGBRegressor trained!
CatBoosting Regressor trained!
AdaBoost Regressor trained!


In [37]:
print('For Test set')
for name, model in models.items():
    predict = model.predict(X_test)
    score = evaluate_model(y_test, predict)
    print(f"{'='*30}")
    print(f"{name}:")
    print(f"MAE: {score[0]}")
    print(f"RMSE: {score[1]}")
    print(f"R2: {score[2]}")
    print(f"{'='*30}")
    print('\n')

For Test set
Linear Regression:
MAE: 4.2253125
RMSE: 5.421371554436202
R2: 0.8792164757750934


Lasso:
MAE: 5.157881810347763
RMSE: 6.519694535667419
R2: 0.8253197323627853


Ridge:
MAE: 4.211100688014259
RMSE: 5.3903870169356365
R2: 0.8805931485028738


K-Neighbors Regressor:
MAE: 5.621
RMSE: 7.253040741647602
R2: 0.7838129945787431


Decision Tree:
MAE: 6.555
RMSE: 8.28039854113315
R2: 0.7182318943495971


Random Forest Regressor:
MAE: 4.665979166666666
RMSE: 5.987577067759639
R2: 0.8526698657410375


XGBRegressor:
MAE: 5.084371652603149
RMSE: 6.5889405377065415
R2: 0.8215894515233413


CatBoosting Regressor:
MAE: 4.612531714976557
RMSE: 6.008631956907363
R2: 0.8516318920747058


AdaBoost Regressor:
MAE: 4.752781843578147
RMSE: 6.0597607864769865
R2: 0.8490961525623404




In [38]:
for name, model in models.items():
    predict = model.predict(X_train)
    score = evaluate_model(y_train, predict)
    print(f"{'='*30}")
    print(f"{name}:")
    print(f"MAE: {score[0]}")
    print(f"RMSE: {score[1]}")
    print(f"R2: {score[2]}")
    print(f"{'='*30}")
    print('\n')

Linear Regression:
MAE: 4.272265625
RMSE: 5.340183842955456
R2: 0.8735068463594741


Lasso:
MAE: 5.206302661246526
RMSE: 6.593815587795566
R2: 0.8071462015863456


Ridge:
MAE: 4.26498782372598
RMSE: 5.323324922741654
R2: 0.8743042615212909


K-Neighbors Regressor:
MAE: 4.516749999999999
RMSE: 5.707683417990174
R2: 0.8554978341651085


Decision Tree:
MAE: 0.01875
RMSE: 0.2795084971874737
R2: 0.9996534669718089


Random Forest Regressor:
MAE: 1.8145452380952383
RMSE: 2.281947184040668
R2: 0.9769024572214803


XGBRegressor:
MAE: 0.6147581052780151
RMSE: 0.9087352405171366
R2: 0.996337063889075


CatBoosting Regressor:
MAE: 2.405393926779502
RMSE: 3.042664195326799
R2: 0.9589358676277713


AdaBoost Regressor:
MAE: 4.790935760489183
RMSE: 5.846390557757516
R2: 0.8483891607320388


